In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Creating a random seed as a good practice for future random calculations
2. Importing the latest version of sklearn
 

In [ ]:
np.random.seed(360)
import sklearn
assert sklearn.__version__>="0.20"

Loading the data into train test splits 

In [ ]:
df_test=pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv')
df_train=pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv')
print("data loading done")

The dummy classifier was the baseline model 


In [ ]:
# from sklearn.dummy import DummyClassifier

# X=pd.DataFrame(df_train)
# dummyclasf=DummyClassifier(strategy="most_frequent")
# print("Dummy Classifier Fitting ")
# dummyclasf.fit(X,y)

# print(dummyclasf.score(X,y))
# temp=dummyclasf.predict(df_test)

# # submission=pd.DataFrame(columns=['id','Made_Purchase'])
# # submission['id']=[i for i in range(len(temp))]
# # submission['Made_Purchase']=temp
# # submission.to_csv('submission.csv', index=False)
# # # DFinal = pd.DataFrame(dummyclasf.predict(df_test))
# # # print(DFinal[0])
# # # DFinal[0].to_csv("submission.csv")


**Data Preprocessing **

In [ ]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder, OrdinalEncoder, MaxAbsScaler, LabelBinarizer, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
import seaborn as sns


In [ ]:
# df_train
# sns.heatmap(df_train[df_train.columns[0:5]])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
temp=df_train['Made_Purchase']
temp.replace({True:1,False:0}, inplace=True)


In [ ]:
plt.pie(df_train.Made_Purchase.value_counts(),labels=['1','0'])
print(df_train.Made_Purchase.value_counts()) #nearly a 60:40 split 

In [ ]:
df_train

In [ ]:
y=pd.DataFrame(df_train.pop("Made_Purchase"))
print(y.shape)


In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train['SeasonalPurchase'].value_counts()

In [ ]:
number_pipeline=Pipeline(steps=[('imputer', KNNImputer(n_neighbors=6)),
                                 ('scaler1', StandardScaler()),
                                 ('scaler2', MinMaxScaler())
                                ])

categorical_pipeline=Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),
                                     ('OneHotEncoding', OneHotEncoder(handle_unknown='ignore'))])

final_pipe=ColumnTransformer(transformers=[('number',number_pipeline,['HomePage','HomePage_Duration','LandingPage','LandingPage_Duration',
                  'ProductDescriptionPage','ProductDescriptionPage_Duration','GoogleMetric:Bounce Rates',
                  'GoogleMetric:Exit Rates','GoogleMetric:Page Values','SeasonalPurchase']),
                                           
                                           ('categories',categorical_pipeline,['Month_SeasonalPurchase','OS','SearchEngine','Zone','Type of Traffic',
                   'CustomerType','Gender','Cookies Setting','Education','Marital Status','WeekendPurchase'])                
]
                            ,remainder='passthrough' )

Train_Features=final_pipe.fit_transform(df_train).toarray()
Test_Features=final_pipe.transform(df_test).toarray()


In [ ]:
# df_train

In [ ]:
# df_test

In [ ]:
Test_Features.shape

In [ ]:
print(Train_Features.shape)
Train_Labels=y.to_numpy()
print(Train_Labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedKFold


cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

X_train,X_test,y_train,y_test=train_test_split(Train_Features, Train_Labels, test_size=0.001,
#                                                stratify=Train_Labels,
                                               random_state=42, shuffle=True)



In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# import xgboost as xgb
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# # xgb= xgb.XGBClassifier(objective="binary:logistic", random_state=42)
# xgb= xgb.XGBClassifier(n_estimators=100)
# xgb.fit(X_train,y_train.ravel())
# y_pred=xgb.predict(X_test)

# print('Testing accuracy',accuracy_score(y_test,y_pred))

# print(classification_report(y_test,y_pred))
# cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
# cm_display.plot()
# plt.show()

In [ ]:
# import xgboost as xgb
# clf_xgb=xgb.XGBClassifier(
#     objective='reg:pseudohubererror', 
#     gamma=0,
#     learning_rate=0.066,
#     max_depth=2,
#     reg_lambda=8.0,
#     scale_pos_weight=1,
#     seed=42,
#     subsample=0.9,
#     colsample_bytree=0.5,
#    eval_metric='aucpr',
#     n_estimators=110)

# clf_xgb.fit(
#     X_train,
#     y_train.ravel())

# y_pred=clf_xgb.predict(X_test)

# print('Testing accuracy',accuracy_score(y_test,y_pred))

# print(classification_report(y_test,y_pred))
# cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
# cm_display.plot()
# plt.show()

In [ ]:
#random forest trees with Xgboost base classifier, 
# extra random trees with Xgboost as base classifier 
from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
clf_xgb=xgb.XGBClassifier(
    objective='reg:pseudohubererror',
    gamma=0,
    learning_rate=0.066,
    max_depth=2,
    reg_lambda=8.0,
   # scale_pos_weight=1,
    seed=42,
    subsample=0.9,
    colsample_bytree=0.5,
   eval_metric='aucpr')
#    n_estimators=110)

# kfold=model_selection.KFold(n_splits=3)

bc=BaggingClassifier(base_estimator=clf_xgb, n_estimators=90, n_jobs=-1, random_state=True)

bc.fit(
    Train_Features,
    Train_Labels.ravel())

y_pred=bc.predict(X_test)

print('Testing accuracy',accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))
cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
cm_display.plot()
plt.show()



In [ ]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.neighbors import KNeighborsClassifier
# import xgboost as xgb
# clf_xgb=xgb.XGBClassifier(
#     objective='reg:pseudohubererror',
#     gamma=0,
#     learning_rate=0.066,
#     max_depth=2,
#     reg_lambda=8.0,
#    # scale_pos_weight=1,
#     seed=42,
#     subsample=0.9,
#     colsample_bytree=0.5,
#    eval_metric='aucpr')
# #    n_estimators=110)


# vclf= VotingClassifier(estimators=[

    
#     ('bc',BaggingClassifier(base_estimator=clf_xgb, n_estimators=90, n_jobs=-1, random_state=True)),
#     ('knn',  KNeighborsClassifier(algorithm= 'kd_tree',leaf_size=200, n_neighbors=100, weights='uniform'))

# ], voting='soft',weights=[1])

# vclf=vclf.fit(X_train,y_train.ravel())
# y_pred = vclf.predict(X_test)
# print(vclf.score(X_test,y_test))
# print('Testing accuracy',accuracy_score(y_test,y_pred))

# print(classification_report(y_test,y_pred))
# cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
# cm_display.plot()
# plt.show()

In [ ]:
# final_predictions=[]
# temp=bc.predict(Test_Features)
# for x in temp:
#     if x==1:
#         final_predictions.append('True')
#     else:
#         final_predictions.append('False')
        
# submission=pd.DataFrame(columns=['id','Made_Purchase'])
# submission['id']=[i for i in range(len(final_predictions))]
# submission['Made_Purchase']=final_predictions
# submission.to_csv('submission.csv', index=False)

In [ ]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# import xgboost as xgb
# vclf= VotingClassifier(estimators=[

#     ('xgb2',xgb.XGBClassifier(
#     objective='reg:pseudohubererror', 
#     gamma=0,
#     learning_rate=0.066,
#     max_depth=2,
#     reg_lambda=8.0,
#     scale_pos_weight=1,
#     seed=42,
#     subsample=0.9,
#     colsample_bytree=0.5,
#    eval_metric='aucpr',
#     n_estimators=110)),
    
#     ('rf',ExtraTreesClassifier(n_estimators=300, min_samples_split=2,random_state=42, max_depth=7))

# ], voting='soft',weights=[1,1])

# vclf=vclf.fit(Train_Features,Train_Labels.ravel())
# y_pred = vclf.predict(X_test)
# print(vclf.score(X_test,y_test))
# print('Testing accuracy',accuracy_score(y_test,y_pred))

# print(classification_report(y_test,y_pred))
# cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
# cm_display.plot()
# plt.show()

In [ ]:
# final_predictions=[]
# temp=vclf.predict(Test_Features)
# for x in temp:
#     if x==1:
#         final_predictions.append('True')
#     else:
#         final_predictions.append('False')
        
# submission=pd.DataFrame(columns=['id','Made_Purchase'])
# submission['id']=[i for i in range(len(final_predictions))]
# submission['Made_Purchase']=final_predictions
# submission.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC, SVC

import xgboost as xgb
clf_xgb=xgb.XGBClassifier(
    objective='reg:pseudohubererror',
    gamma=0,
    learning_rate=0.066,
    max_depth=2,
    reg_lambda=8.0,
   # scale_pos_weight=1,
    seed=42,
    subsample=0.9,
    colsample_bytree=0.5,
   eval_metric='aucpr')

estimators=[
    
  ('bc',BaggingClassifier(base_estimator=clf_xgb, n_estimators=90, n_jobs=-1, random_state=True)),
#  ('knn',  KNeighborsClassifier(algorithm= 'kd_tree',leaf_size=200, n_neighbors=100, weights='uniform')),
  ('rf',ExtraTreesClassifier(n_estimators=200, min_samples_split=2,random_state=42)),
  #('rf1', RandomForestClassifier(n_estimators=200, min_samples_split=2,random_state=42))
    
]

fclf=StackingClassifier(
   estimators=estimators, final_estimator=SVC(gamma='scale', kernel='rbf', C=800)
    
)

fclf=fclf.fit(X_train,y_train.ravel())
y_pred = fclf.predict(X_test)
print(fclf.score(X_test,y_test))
print('Testing accuracy',accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))
cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
stop

In [ ]:
# final_predictions=[]
# temp=fclf.predict(Test_Features)
# for x in temp:
#     if x==1:
#         final_predictions.append('True')
#     else:
#         final_predictions.append('False')
        
# submission=pd.DataFrame(columns=['id','Made_Purchase'])
# submission['id']=[i for i in range(len(final_predictions))]
# submission['Made_Purchase']=final_predictions
# submission.to_csv('submission.csv', index=False)

In [ ]:
# print(len(final_predictions))
# print(final_predictions)

In [ ]:
stop

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(n_estimators=500, random_state=0)

etc=etc.fit(X_train,y_train.ravel())
y_pred = etc.predict(X_test)
print(etc.score(X_test,y_test))
print('Testing accuracy',accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))
cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
cm_display.plot()
plt.show()

In [ ]:
# final_predictions=[]
# temp=vclf.predict(Test_Features)
# for x in temp:
#     if x==1:
#         final_predictions.append('True')
#     else:
#         final_predictions.append('False')
        
# submission=pd.DataFrame(columns=['id','Made_Purchase'])
# submission['id']=[i for i in range(len(final_predictions))]
# submission['Made_Purchase']=final_predictions
# submission.to_csv('submission.csv', index=False)

In [ ]:
# print(len(final_predictions))
# print(final_predictions)

In [ ]:
stop

In [ ]:
# from sklearn.neural_network import MLPClassifier

# mlp=MLPClassifier(hidden_layer_sizes=(88,70,40,11,5,2),max_iter=500, activation='relu',early_stopping=True)
# mlp.fit(Train_Features,Train_Labels.ravel())
# y_pred=mlp.predict(X_test)

# print(mlp.score(X_test,y_test))


In [ ]:
from sklearn.linear_model import LogisticRegression, SGDRegressor
# from sklearn.model_selection import GridSearchCV

# param_grid={
    
#     'logistic__penalty':['l2'],
#     'logistic__C':[1,2,3,4,5,6,7,8,9,10],
#     'logistic__tol':[1e-3,1e-2]
    
# }

# pipeline=Pipeline([
    
#     ("logistic",LogisticRegression(max_iter=10000,random_state=42))
    
# ])

# gs=GridSearchCV(estimator=pipeline,cv=cv,param_grid=param_grid,return_train_score=True, n_jobs=-1)


# results=gs.fit(X_train,y_train.ravel())
# print(results.score(X_test,y_test))
# print(results.best_params_)

In [ ]:
# #1 Naive Bayes optimal hyperparameters 
# from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB

# pipelineNB=Pipeline([
#     ("naive_bayes", BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))   
# ])

# results=pipelineNB.fit(X_train,y_train.ravel())
# print(results.score(X_test,y_test))


In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("BernoulliNB.txt", "w")
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

In [ ]:
# from sklearn.svm import SVC

# svc=SVC(C=1,kernel='poly',gamma='auto',tol=1e-5,decision_function_shape='ovo',cache_size=7000,degree=1)

# results=svc.fit(X_train,y_train.ravel())

In [ ]:
# print(results.score(X_test,y_test))

In [ ]:
# #2 SVM optimal hyperparameters
# from sklearn.model_selection import GridSearchCV

# from sklearn.svm import SVC

# param_grid={
    
#     'SVC__C':[1,2],
#     'SVC__kernel':['linear'],
#     'SVC__gamma':['auto','scale'],
#     'SVC__degree':[1,2,3,4,5,6,7],
#     'SVC__decision_function_shape':['ovo','ovr']
       
# }

# pipelineSVC=Pipeline([
#     ("SVC", SVC(cache_size=7000))   
# ])

# gs=GridSearchCV(estimator=pipelineSVC,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1,verbose=10)


# results=gs.fit(X_train,y_train.ravel())
# print(results.score(X_test,y_test))
# print(results.best_params_)

In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("SVC_linear.txt", "w")
# txtf.write("Best Parameters:   %s\n" % results.best_params_)
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

In [ ]:
# #2 SVM optimal hyperparameters
# from sklearn.model_selection import GridSearchCV

# from sklearn.svm import SVC

# param_grid={
    
#     'SVC__C':[1,2],
#     'SVC__kernel':['rbf'],
#     'SVC__gamma':['auto','scale'],
#     'SVC__degree':[1,2,3,4,5,6,7],
#     'SVC__decision_function_shape':['ovo','ovr']
       
# }

# pipelineSVC=Pipeline([
#     ("SVC", SVC(cache_size=7000))   
# ])

# gs=GridSearchCV(estimator=pipelineSVC,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1,verbose=10)


# results=gs.fit(X_train,y_train.ravel())
# print(results.score(X_test,y_test))
# print(results.best_params_)

In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("SVC_rbf.txt", "w")
# txtf.write("Best Parameters:   %s\n" % results.best_params_)
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

In [ ]:
# #2 SVM optimal hyperparameters
# from sklearn.model_selection import GridSearchCV

# from sklearn.svm import SVC

# param_grid={
    
#     'SVC__C':[1,2],
#     'SVC__kernel':['poly'],
#     'SVC__gamma':['auto','scale'],
#     'SVC__degree':[1,2,3,4,5,6,7],
#     'SVC__decision_function_shape':['ovo','ovr']
       
# }

# pipelineSVC=Pipeline([
#     ("SVC", SVC(cache_size=7000))   
# ])

# gs=GridSearchCV(estimator=pipelineSVC,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1,verbose=10)


# results=gs.fit(X_train,y_train.ravel())
# print(results.score(X_test,y_test))
# print(results.best_params_)

In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("SVC_poly.txt", "w")
# txtf.write("Best Parameters:   %s\n" % results.best_params_)
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

In [ ]:
# #2 SVM optimal hyperparameters
# from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

# param_grid={
    
#     'SVC__C':[1,2],
#     'SVC__kernel':['sigmoid'],
#     'SVC__gamma':['auto','scale'],
#     'SVC__degree':[1,2,3,4,5,6,7],
#     'SVC__decision_function_shape':['ovo','ovr']
       
# }

# pipelineSVC=Pipeline([
#     ("SVC", SVC(cache_size=7000))   
# ])

# gs=GridSearchCV(estimator=pipelineSVC,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1,verbose=10)


# results=gs.fit(X_train,y_train.ravel())
# print(results.score(X_test,y_test))
# print(results.best_params_)

In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("SVC_sigmoid.txt", "w")
# txtf.write("Best Parameters:   %s\n" % results.best_params_)
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

In [ ]:
# SVC().get_params().keys()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV
# param_grid={
    
#     'KNN__n_neighbors':np.arange(1,300,10),
#     'KNN__weights':['uniform'],
#     #'KNN__algorithm':['ball_tree','kd_tree','brute'],
#     'KNN__algorithm':['kd_tree','brute'],
#     'KNN__leaf_size':[30,40,50,60,70,80,90,100,110,120,130,140]
       
# }

# pipelineKNN=Pipeline([
#     ("KNN", KNeighborsClassifier())   
# ])

# gs=GridSearchCV(estimator=pipelineKNN,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1,scoring='accuracy',verbose=10)
# results=gs.fit(X_train,y_train.ravel())
# print(results.best_params_)

In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("KNN.txt", "w")
# txtf.write("Best Parameters:   %s\n" % results.best_params_)
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

Best Parameters:   {'KNN__algorithm': 'kd_tree',  
'KNN__leaf_size': 100,  
'KNN__n_neighbors': 41,  
'KNN__weights': 'uniform'}  

R2 score:   0.6362402443162538

In [ ]:
# print(results.score(X_test,y_test))

In [ ]:
# #3 adaboost 
from sklearn.ensemble import AdaBoostClassifier
# from sklearn.model_selection import GridSearchCV

# param_grid={
    
#     'Ada__n_estimators':[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200],
#     'Ada__learning_rate':[0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5]
# }


# pipelineAda=Pipeline([
#     ("Ada", AdaBoostClassifier(random_state=42))   
# ])

# gs=GridSearchCV(estimator=pipelineAda,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1, verbose=10)
# results=gs.fit(X_train,y_train.ravel())
# print(results.best_params_)


In [ ]:
# print(results.score(X_test,y_test))
# txtf = open("Adaboost.txt", "w")
# txtf.write("Best Parameters:   %s\n" % results.best_params_)
# txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
# txtf.close()

Best Parameters:   {'Ada__learning_rate': 1.3, 
'Ada__n_estimators': 10}  

R2 score:   0.6599932134373939


In [ ]:
# from sklearn.ensemble import RandomForestClassifier 
# from sklearn.model_selection import GridSearchCV

# param_grid={
    
#      'rf__n_estimators':np.arange(1,1000,100),
#      'rf__max_depth':np.arange(1,1000,100),
#      'rf__min_samples_split':np.arange(1,4,1),
#      'rf__criterion':['gini','entropy','log_loss']
# }


# pipelinerf=Pipeline([
#      ("rf", RandomForestClassifier(random_state=42))   
#  ])

# gs=GridSearchCV(estimator=pipelinerf,cv=2,param_grid=param_grid,return_train_score=True, n_jobs=-1, verbose=10)

# gs.fit(Train_Features,Train_Labels.ravel())
# results=gs.score(X_test,y_test)
# print(results)



In [ ]:
from sklearn.ensemble import RandomForestClassifier 

rf=RandomForestClassifier(n_estimators=200, min_samples_split=2,random_state=42, max_depth=7)
rf.fit(Train_Features,Train_Labels.ravel())

print(rf.score(X_test,y_test))

y_pred=rf.predict(X_test)
print('Testing accuracy',accuracy_score(y_test,y_pred))
y_train_pred=rf.predict(X_train)
print('Training accuracy',accuracy_score(y_train_pred,y_train))

print(classification_report(y_test,y_pred))
cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
cm_display.plot()
plt.show()
# y_pred=clf.predict(X_test)
# print(f1_score(y_test,y_pred))

In [ ]:
# final_predictions=[]
# temp=rf.predict(Test_Features)
# for x in temp:
#     if x==1:
#         final_predictions.append('True')
#     else:
#         final_predictions.append('False')
        
# submission=pd.DataFrame(columns=['id','Made_Purchase'])
# submission['id']=[i for i in range(len(final_predictions))]
# submission['Made_Purchase']=final_predictions
# submission.to_csv('submission.csv', index=False)

In [ ]:
# print(final_predictions)

In [ ]:
import xgboost as xgb

clf_xgb=xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42,eval_metric='aucpr')
clf_xgb.fit(
    X_train,
    y_train,
    verbose=True,
    early_stopping_rounds=10,
    
    eval_set=[(X_test,y_test)]

)

In [ ]:
# clf_xgb=xgb.XGBClassifier(
#     objective='binary:logistic', 
#     gamma=0,
#     learning_rate=0.01,
#     max_depth=2,
#     reg_lambda=7.9,
#     scale_pos_weight=2,
#     seed=42,
# #     subsample=0.9,
# #     colsample_bytree=0.5,
# #    eval_metric='aucpr',
#     n_estimators=1000)

# clf_xgb.fit(
#     X_train,
#     y_train.ravel())

# y_pred=clf_xgb.predict(X_test)

# print('Testing accuracy',accuracy_score(y_test,y_pred))

# print(classification_report(y_test,y_pred))
# cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
# cm_display.plot()
# plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
param_grid={
    
    'max_depth':[1,2,3,4],
    'learning_rate':[0.04,0.05,0.06,0.07,0.08],
    'gamma':[0,0.001],
    'reg_lambda':[7.0,8.0],
    'scale_pos_weight':[1,2,3]
}

optimal_params=GridSearchCV( 
                
             estimator=xgb.XGBRFClassifier(objective='reg:pseudohubererror',
                                        seed=42,
                                        subsample=0.9,
                                        colsample_bytree=0.5),
    param_grid=param_grid,
    scoring='roc_auc',
    verbose=0,
    n_jobs=10,
    cv=3
)

optimal_params.fit(X_train,
                  y_train,
                  eval_metric='auc',
                  eval_set=[(X_test,y_test)],
                  verbose=False)
print(optimal_params.best_params_)



y_pred=optimal_params.predict(X_test)

print('Testing accuracy',accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))
cm_display=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,y_pred), display_labels=[0,1])
cm_display.plot()
plt.show()


Pseudo huberloss [best params]  
{'gamma': 0, 'learning_rate': 0.066, 'max_depth': 2, 'reg_lambda': 8.0, 'scale_pos_weight': 1}  

Testing accuracy 0.6830675262979301


In [ ]:
stop

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier

estimators=[
   # ('svc', SVC(C=2, decision_function_shape='ovo', degree=1, gamma='auto', kernel='rbf',probability=True)),
     #('bernb', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)),
   #  ('knn',  KNeighborsClassifier(algorithm= 'kd_tree',leaf_size=100, n_neighbors=41, weights='uniform')),
     #('ada', AdaBoostClassifier(learning_rate=1.3, n_estimators=10)),
     ('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=42)),
     ('rf' , RandomForestClassifier(n_estimators=5, min_samples_split=2,random_state=42, max_depth=10))
]

fclf=StackingClassifier(
    estimators=estimators, final_estimator=SGDClassifier() # Default hinge loss
)
fclf.fit(Train_Features,Train_Labels.ravel())
fclf.score(X_test,y_test)
print(results.best_params_)



In [ ]:
txtf = open("rf.txt", "w")
txtf.write("Best Parameters:   %s\n" % results.best_params_)
txtf.write("R2 score:   %s\n\n" % results.score(X_test,y_test))
txtf.close()

In [ ]:
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
vclf= VotingClassifier(estimators=[

    # ('svc', SVC(C=2, decision_function_shape='ovo', degree=1, gamma='auto', kernel='rbf',probability=True)),
     ('bernb', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)),
     ('knn',  KNeighborsClassifier(algorithm= 'kd_tree',leaf_size=100, n_neighbors=41, weights='uniform')),
     #('ada', AdaBoostClassifier(learning_rate=1.3, n_estimators=10)),
     ('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=42)),
     ('rf' , RandomForestClassifier(n_estimators=5, min_samples_split=2,random_state=42, max_depth=10)),
     ( 'mlp', MLPClassifier(hidden_layer_sizes=(88,70,40,11,5,2),max_iter=500, activation='relu',early_stopping=True))
    

], voting='soft')

vclf=vclf.fit(Train_Features,Train_Labels.ravel())
y_pred = vclf.predict(X_test)
print(vclf.score(X_test,y_test))

In [ ]:
from sklearn.metrics import accuracy_score
# using accuracy_score
score = accuracy_score(y_test, y_pred)                    
print(score)
# fclf.score(X_test,y_test)